# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
loaded_csv_file = pd.read_csv("../output_data/city_weather.csv")
loaded_csv_file.head()


,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
0,4035715,Avarua,CK,-21.2078,-159.7750,82.40,78,75,5.75
1,2391893,Savé,BJ,8.0425,2.4860,79.65,74,71,5.19
2,2206939,Bluff,NZ,-46.6000,168.3333,73.00,56,53,5.99
3,2120047,Ust'-Omchug,RU,61.1500,149.6333,-13.11,93,99,4.56
4,2645198,Kirkwall,GB,58.9848,-2.9587,35.60,80,40,7.83


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [5]:
#humidity heatmap
locations = loaded_csv_file[["Lat", "Lng"]]
humidity = loaded_csv_file["Humidity"]

#heatmap plotting
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 600
heat_layer.point_radius = 4


fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create new DataFrame fitting weather criteria
# max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

new_city_data = loaded_csv_file.loc[(loaded_csv_file["Temperature"] < 80) \
                                    & (loaded_csv_file["Temperature"] > 70) \
                                    & (loaded_csv_file["Wind Speed"] < 10) \
                                    & (loaded_csv_file["Clouds"] == 0)].dropna()
new_city_data

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed
12,2208248,Kaitangata,NZ,-46.2817,169.8464,72.00,60,0,1.01
20,2164457,Grenfell,AU,-33.9000,148.1667,72.05,25,0,7.02
24,1661950,Ban Nahin,LA,18.2418,104.2122,74.88,60,0,2.10
156,3430443,Necochea,AR,-38.5473,-58.7368,73.99,75,0,4.00
206,1259385,Port Blair,IN,11.6667,92.7500,77.54,64,0,7.67
225,3981391,Tomatlán,MX,19.9333,-105.2500,72.90,79,0,3.15
250,935215,Saint-Philippe,RE,-21.3585,55.7679,70.61,64,0,5.75
317,3347019,Mossamedes,AO,-15.1961,12.1522,72.09,83,0,3.94
357,935214,Saint-Pierre,RE,-21.3393,55.4781,72.72,64,0,5.75
371,4026082,Lázaro Cárdenas,MX,17.9583,-102.2000,75.27,74,0,4.47


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# create dataframe called hotel_df with hotel name column
hotel_df = new_city_data
hotel_df["Hotel Name"] = np.nan
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
12,2208248,Kaitangata,NZ,-46.2817,169.8464,72.00,60,0,1.01,NaN
20,2164457,Grenfell,AU,-33.9000,148.1667,72.05,25,0,7.02,NaN
24,1661950,Ban Nahin,LA,18.2418,104.2122,74.88,60,0,2.10,NaN
156,3430443,Necochea,AR,-38.5473,-58.7368,73.99,75,0,4.00,NaN
206,1259385,Port Blair,IN,11.6667,92.7500,77.54,64,0,7.67,NaN
225,3981391,Tomatlán,MX,19.9333,-105.2500,72.90,79,0,3.15,NaN
250,935215,Saint-Philippe,RE,-21.3585,55.7679,70.61,64,0,5.75,NaN
317,3347019,Mossamedes,AO,-15.1961,12.1522,72.09,83,0,3.94,NaN
357,935214,Saint-Pierre,RE,-21.3393,55.4781,72.72,64,0,5.75,NaN
371,4026082,Lázaro Cárdenas,MX,17.9583,-102.2000,75.27,74,0,4.47,NaN


In [8]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City ID,City,Country,Lat,Lng,Temperature,Humidity,Clouds,Wind Speed,Hotel Name
12,2208248,Kaitangata,NZ,-46.2817,169.8464,72.00,60,0,1.01,NaN
20,2164457,Grenfell,AU,-33.9000,148.1667,72.05,25,0,7.02,NaN
24,1661950,Ban Nahin,LA,18.2418,104.2122,74.88,60,0,2.10,NaN
156,3430443,Necochea,AR,-38.5473,-58.7368,73.99,75,0,4.00,NaN
206,1259385,Port Blair,IN,11.6667,92.7500,77.54,64,0,7.67,NaN
225,3981391,Tomatlán,MX,19.9333,-105.2500,72.90,79,0,3.15,NaN
250,935215,Saint-Philippe,RE,-21.3585,55.7679,70.61,64,0,5.75,NaN
317,3347019,Mossamedes,AO,-15.1961,12.1522,72.09,83,0,3.94,NaN
357,935214,Saint-Pierre,RE,-21.3393,55.4781,72.72,64,0,5.75,NaN
371,4026082,Lázaro Cárdenas,MX,17.9583,-102.2000,75.27,74,0,4.47,NaN


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# # Add marker layer ontop of heat map
# marker = gmaps.marker(locations, info_box_content=hotel_info)
# fig.add_layer(marker)

# # Display figure
# fig

# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))